# Facial Emotion Recognition 

A baby is able to recognize it's parent's faces when it is about a couple of weeks old. The baby grows and matures and by the time it is about two months old, it starts to display social cues. It is able to understand basic emotions like a smile. 

This project involves training a neural network to do just that, recognize facial emotions. We owe our ability to do that to the wonderful process of evolution. Replicating this is certainly not easy but possible.

For this project, I will be using [Google Collab](/https://colab.research.google.com/). For those who are unaware, it is a research project created to disseminate ML education and you can train your neural networks on their Tesla K80 GPUs. It sure speeds up the entire training process.


## 1.  Connect google Drive.

Once connected, we will be able to access the FER2013 dataset.   

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## 2. Import Libraries.

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Sequential #Initialise our neural network model as a sequential network
from keras.layers import Conv2D #Convolution operation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation#Applies activation function
from keras.layers import Dropout#Prevents overfitting by randomly converting few outputs to zero
from keras.layers import MaxPooling2D # Maxpooling function
from keras.layers import Flatten # Converting 2D arrays into a 1D linear vector
from keras.layers import Dense # Regular fully connected neural network
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


## 3. Define Data loading mechanism.


the* load_data()* access the [FER2013](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data) dataset from Google Drive and then we start reading it line by line. Our dataset has 35,887 images, which are classified into seven categories -  Angry, Disgust, Fear, Happy, Sad, Surprsie, and Neutral. Let us have a look at the file.

![Dataset overview](Assets\TrainingData.png) 

We are only going to need the Class and the Image Data. Image data is one big string of numbers exactly 2304 numbers. These are pixel intensity values and each number represents the darkness of that pixel in the image. It can take any value from 0 (White) to 255 (Black). We split the string to get hold of individual numbers and then reshape it into a 48 x 48 array and dividing by 255 normalizes the data. 

Once we go through all the images, we expand the dimension of our data array by one to accommodate for the channel value. We one-hot encode the labels then return the numpy arrays.     


In [0]:
def load_data(dataset_path):

  data = []
  labels =[]

  with open(dataset_path, 'r') as file:
      for line_no, line in enumerate(file.readlines()):
          if 0 < line_no <= 35887:
            curr_class, line, set_type = line.split(',')
            image_data = np.asarray([int(x) for x in line.split()]).reshape(48, 48)#Creating a list out of the string then converting it into a 2-Dimensional numpy array.
            image_data =image_data.astype(np.uint8)/255.0
            data.append(image_data)
            labels.append(curr_class)
      data = np.expand_dims(data, -1)   
      labels = to_categorical(labels, num_classes = 7)
    
      return np.array(data), np.array(labels)

## 4. Load the data.


In [5]:
seed = 7
test_size = 0.1
dataset_path = "/content/gdrive/My Drive/Colab Notebooks/Emotion Recognition/Data/fer2013.csv" 
data, labels = load_data(dataset_path)
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = test_size,random_state = seed)

print("Number of images in Training set:", len(train_data))
print("Number of images in Test set:", len(test_data))

Number of images in Training set: 32298
Number of images in Test set: 3589


## 5. Define and Deploy the neural network.

We have built ourselves a Convolutional Neural Network which has convolutional layers armed with filters that extract features out of images. 

Callback functions are those functions which are called at the end of every epoch and in this model, we use two callback functions - ReduceLROnPlateau and EarlyStopping. 



> **1. ReduceLROnPlateau:** monitors a certain variable, in this case, validation loss and alters the learning rate when the value stops significantly changing after some certain number of epochs (patience).

>  **2. EarlyStopping:** At times our optimiser can land in *local optima* and get stuck there. There is no point in continuing the training as there won't be any further improvements. 

> **3. ModelCheckpoin**t: Saves the best version of our model along with the weights, so that in case any crash occurs, our model can be recovered.

![Network Architecture](Assets\NetworkArchitecture.png)

We have used Keras to deploy our neural network and if you look at the architecture then you will notice that we have used Dropout layers frequently. Dropout layers inhibit overfitting by randomly dropping out units from the neural network. We will use 20 percent of the training data as validation data.





In [6]:
#######HYPERPARAMATERS###########
epochs = 100
batch_size = 64
learning_rate = 0.001
#################################

model = Sequential()
    
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01)))
model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))
    
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
    
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
    
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
    
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
adam = optimizers.Adam(lr = learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=6, mode='auto')
checkpointer = ModelCheckpoint('/content/gdrive/My Drive/Colab Notebooks/Emotion Recognition/Model/Train_weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.2,
          shuffle = True,
          callbacks=[lr_reducer, checkpointer, early_stopper]
          )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 23, 128)       512       
__________

## 6. Test the model.

We trained the neural network and we achieved the highest validation accuracy of 62.23%. Now, we will use the test data to check how well our model generalizes. 

We score an astounding 60.2 % on the test set. Though we barely, made it to the first division, we still made it the 12th rank of the Kaggle competition.

In [7]:
predicted_test_labels = np.argmax(model.predict(test_data), axis=1)
test_labels = np.argmax(test_labels, axis=1)
print ("Accuracy score = ", accuracy_score(test_labels, predicted_test_labels))

Accuracy score =  0.6015603232098078


## 7. Save the model.

In [8]:
from keras.models import model_from_json
model_json = model.to_json()
with open("/content/gdrive/My Drive/Colab Notebooks/Emotion Recognition/Models/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/gdrive/My Drive/Colab Notebooks/Emotion Recognition/Models/model.h5")
print("Saved model to disk")

Saved model to disk


## 8. Up next?

Our neural network is trained and we are ready to use it in an application. In the accompanying IPython Notebook, we will notice an application which analyzes your webcam feed and identifies faces and then recognizes the emotions.

